# Analyzing Health and Nutrition Data

We'll start by importing relevant _libraries_ (i.e. prebuilt chunks of Python that have useful functions).

In [1]:
import pandas as pd    #data manipulation
import numpy as np     #mathematical operations

import matplotlib.pyplot as plt      #plotting tools

from urllib.request import urlopen   #web url reading
import json                          #json reader

! pip install xport
import xport                   #xport reader (us gov't data export format)

## Exploratory Data Analysis

### Demographics

We'll be importing data from the [National Health and Nutrition Examination Survey](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2017) as a pandas dataframe.  We can start by looking at the underlying demographics of the survey participants.

In [10]:
# download_url = "https://github.com/annahaensch/DataAndSocialJustice/blob/main/Data/Health_and_Nutrition/DEMO_I.XPT?raw=true"

url = "DEMO_I.XPT"

with open(url, 'rb') as f:
    df = xport.to_dataframe(f)

In [4]:
df.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGE,DMDHRBR4,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,83732.0,9.0,2.0,1.0,62.0,NaN,3.0,3.0,1.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,5.0,1.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,1.0,1.0,62.0,1.0,5.0,1.0,3.0,134671.370419,135629.507405,1.0,125.0,10.0,10.0,4.39
1,83733.0,9.0,2.0,1.0,53.0,NaN,3.0,3.0,1.0,NaN,2.0,NaN,2.0,2.0,7.0,NaN,3.0,3.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,53.0,2.0,3.0,3.0,NaN,24328.560239,25282.425927,1.0,125.0,4.0,4.0,1.32
2,83734.0,9.0,2.0,1.0,78.0,NaN,3.0,3.0,2.0,NaN,1.0,2.0,1.0,1.0,NaN,NaN,3.0,1.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,NaN,2.0,2.0,0.0,0.0,2.0,2.0,79.0,1.0,3.0,1.0,3.0,12400.008522,12575.838818,1.0,131.0,5.0,5.0,1.51
3,83735.0,9.0,2.0,2.0,56.0,NaN,3.0,3.0,2.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,5.0,6.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,56.0,1.0,5.0,6.0,NaN,102717.995647,102078.634508,1.0,131.0,10.0,10.0,5.00
4,83736.0,9.0,2.0,2.0,42.0,NaN,4.0,4.0,2.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,4.0,3.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,0.0,2.0,0.0,2.0,42.0,1.0,4.0,3.0,NaN,17627.674984,18234.736219,2.0,126.0,7.0,7.0,1.23


To understand the column headings here, we should consult the [NHANES documentation brochure](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm).  Who is missing from this data and why?  How should that impact our analysis?  We can look at the summary statistics of the dataframe with:

In [5]:
df.describe()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGE,DMDHRBR4,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,9971.000000,9971.0,9971.000000,9971.000000,9971.000000,695.000000,9971.000000,9971.000000,9544.000000,4060.000000,6149.000000,527.000000,9971.000000,9969.000000,2236.000000,2647.000000,5719.000000,5719.000000,1288.000000,9971.000000,9970.000000,9971.000000,9642.000000,9642.000000,9642.000000,6977.000000,6978.000000,6978.000000,5962.000000,9971.000000,9971.000000,9971.000000,9971.000000,9971.000000,9971.000000,9971.000000,9575.000000,9575.000000,9909.000000,5226.000000,9971.000000,9971.000000,9971.000000,9971.000000,9626.000000,9642.000000,8919.000000
mean,88717.000000,9.0,1.957176,1.509377,31.899408,10.785612,3.007321,3.213118,1.518650,104.534975,1.914295,1.529412,1.243907,1.128197,8.751789,5.988666,3.439587,2.650988,2.017857,1.139103,1.629990,1.954167,1.125700,1.999067,1.957996,1.085280,1.991831,1.950416,1.142570,3.892990,3.772641,0.554207,1.059673,0.404674,1.493230,46.178819,1.415561,3.459530,2.813503,3.596441,31740.150837,31740.150837,1.485809,126.266673,11.522439,11.252126,2.269277
std,2878.524101,0.0,0.202471,0.499937,24.768829,7.015325,1.317594,1.679606,0.499678,68.972946,0.279951,0.603056,1.446152,0.422122,17.834753,5.873498,1.314679,2.671713,0.355432,0.346072,0.482831,0.209133,0.331528,0.030539,0.200608,0.279318,0.090016,0.217100,0.397276,1.745113,1.808749,0.827284,1.238771,0.704488,0.499979,15.828729,2.681659,1.331829,5.962991,1.381068,32929.536529,34105.572345,0.499824,4.238500,15.191049,15.219413,1.576345
min,83732.000000,9.0,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,18.000000,1.000000,1.000000,1.000000,1.000000,3293.928267,0.000000,1.000000,119.000000,1.000000,1.000000,0.000000
25%,86224.500000,9.0,2.000000,1.000000,9.000000,5.000000,2.000000,2.000000,1.000000,41.000000,2.000000,1.000000,1.000000,1.000000,3.000000,2.000000,3.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,34.000000,1.000000,3.000000,1.000000,3.000000,12878.501052,12550.531753,1.000000,123.000000,6.000000,5.000000,0.970000
50%,88717.000000,9.0,2.000000,2.000000,27.000000,10.000000,3.000000,3.000000,2.000000,100.000000,2.000000,2.000000,1.000000,1.000000,5.000000,5.000000,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,4.000000,4.000000,0.000000,1.000000,0.000000,1.000000,44.000000,1.000000,4.000000,1.000000,4.000000,20160.468137,20281.319287,1.000000,126.000000,8.000000,8.000000,1.820000
75%,91209.500000,9.0,2.000000,2.000000,53.000000,17.000000,4.000000,4.000000,2.000000,162.000000,2.000000,2.000000,1.000000,1.000000,7.000000,9.000000,4.000000,5.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,5.000000,5.000000,1.000000,2.000000,1.000000,2.000000,57.000000,2.000000,4.000000,4.000000,5.000000,33257.364572,33708.148362,2.000000,130.000000,14.000000,14.000000,3.480000
max,93702.000000,9.0,2.000000,2.000000,80.000000,24.000000,5.000000,7.000000,2.000000,239.000000,2.000000,7.000000,99.000000,9.000000,99.000000,99.000000,9.000000,99.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,7.000000,7.000000,3.000000,4.000000,3.000000,2.000000,80.000000,99.000000,9.000000,99.000000,9.000000,233755.8418

For any individual column we can get a closer look at the value counts with:

In [6]:
df["RIAGENDR"].value_counts()

2.0    5079
1.0    4892
Name: RIAGENDR, dtype: int64

### Dietary Data

We'll be importing dietary survey data from the [National Health and Nutrition Examination Survey](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2017) as a pandas dataframe.  To understand the data we can look in the accompanying [documentation brochure](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DR2IFF_I.htm).

In [13]:
download_url = "https://github.com/annahaensch/DataAndSocialJustice/blob/main/Data/Health_and_Nutrition/DR2IFF_I.XPT?raw=true"

url = "DR2IFF_I.XPT"

with open(url, 'rb') as f:
    df_diet = xport.to_dataframe(f)

In [9]:
df_diet

,SEQN,WTDRD1,WTDR2D,DR2ILINE,DR2DRSTZ,DR2EXMER,DRABF,DRDINT,DR2DBIH,DR2DAY,DR2LANG,DR2CCMNM,DR2CCMTX,DR2_020,DR2_030Z,DR2FS,DR2_040Z,DR2IFDCD,DR2IGRMS,DR2IKCAL,DR2IPROT,DR2ICARB,DR2ISUGR,DR2IFIBE,DR2ITFAT,DR2ISFAT,DR2IMFAT,DR2IPFAT,DR2ICHOL,DR2IATOC,DR2IATOA,DR2IRET,DR2IVARA,DR2IACAR,DR2IBCAR,DR2ICRYP,DR2ILYCO,DR2ILZ,DR2IVB1,DR2IVB2,...,DR2IFF,DR2IFDFE,DR2ICHL,DR2IVB12,DR2IB12A,DR2IVC,DR2IVD,DR2IVK,DR2ICALC,DR2IPHOS,DR2IMAGN,DR2IIRON,DR2IZINC,DR2ICOPP,DR2ISODI,DR2IPOTA,DR2ISELE,DR2ICAFF,DR2ITHEO,DR2IALCO,DR2IMOIS,DR2IS040,DR2IS060,DR2IS080,DR2IS100,DR2IS120,DR2IS140,DR2IS160,DR2IS180,DR2IM161,DR2IM181,DR2IM201,DR2IM221,DR2IP182,DR2IP183,DR2IP184,DR2IP204,DR2IP205,DR2IP225,DR2IP226
0,83732.0,92670.699919,69945.934107,1.0,1.0,87.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,18000.0,7.0,1.0,1.0,92101000.0,255.00,3.0,0.31,0.00,0.00,0.0,0.05,0.005,0.038,0.003,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036,0.194,...,5.0,5.0,6.6,0.00,0.0,0.0,0.0,0.3,5.0,8.0,8.0,0.03,0.05,0.005,5.0,125.0,0.0,102.0,0.0,0.0,253.45,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.0,0.000,0.000,0.000,0.000
1,83732.0,92670.699919,69945.934107,2.0,1.0,87.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,18000.0,7.0,1.0,1.0,91200040.0,2.00,7.0,0.02,1.78,1.70,0.0,0.00,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000,9.0,0.0,0.0,0.0,0.0,0.0,0.18,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000
2,83732.0,92670.699919,69945.934107,3.0,1.0,87.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,18000.0,7.0,1.0,1.0,12210400.0,1.96,10.0,0.05,1.16,0.15,0.0,0.65,0.375,0.192,0.008,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.01,0.00,0.000,2.0,13.0,0.0,0.0,0.0,0.0,0.06,0.000,0.002,0.022,0.017,0.132,0.052,0.062,0.084,0.000,0.191,0.000,0.000,0.007,0.001,0.0,0.000,0.000,0.000,0.000
3,83732.0,92670.699919,69945.934107,4.0,1.0,87.0,2.0,2.0,2.0,4.0,1.0,0.0,0.0,18000.0,7.0,NaN,1.0,94000100.0,240.00,0.0,0.00,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,7.0,0.0,2.0,0.00,0.02,0.024,10.0,0.0,0.0,0.0,0.0,0.0,239.76,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000
4,83732.0,92670.699919,69945.934107,5.0,1.0,87.0,2.0,2.0,2.0,4.0,1.0,0.0,0.0,26100.0,1.0,7.0,2.0,58100010.0,222.00,536.0,25.27,38.23,5.10,2.8,30.90,10.758,10.965,6.869,320.0,1.91,0.0,168.0,173.0,0.0,55.0,7.0,943.0,417.0,0.451,0.679,...,39.0,125.0,189.9,1.02,0.0,0.3,2.3,12.4,286.0,429.0,38.0,4.43,2.63,0.167,1297.0,382.0,48.0,0.0,0.0,0.0,122.60,0.155,0.044,0.056,0.102,0.078,0.699,6.252,2.959,0.475,10.223,0.158,0.002,5.934,0.544,0.0,0.178,0.000,0.013,0.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100675,93702.0,67560.380806,55166.938286,13.0,1.0,87.0,2.0,2.0,4.0,3.0,1.0,0.0,0.0,70200.0,3.0,1.0,1.0,26153120.0,76.00,126.0,22.69,0.07,0.03,0.0,3.25,0.543,1.278,1.287,36.0,0.55,0.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.099,0.102,...,2.0,2.0,60.5,1.74,0.0,0.4,1.6,3.4,4.0,259.0,33.0,0.72,0.35,0.034,280.0,411.0,84.2,0.0,0.0,0.0,48.37,0.000,0.000,0.000,0.000,0.001,0.008,0.371,0.131,0.021,1.224,0.019,0.001,1.027,0.144,0.0,0.014,0.011,0.004,0.082
100676,93702.0,67560.380806,55166.938286,15.0,1.0,87.0,2.0,2.0,4.0,3.0,1.0,2.0,4.0,70200.0,3.0,1.0,1.0,75113000.0,35.00,5.0,0.32,1.04,0.69,0.4,0.05,0.006,0.002,0.026,0.0,0.06,0.0,0.0,9.0,1.0,105.0,0.0,0.0,97.0,0.014,0.009,...,10.0,10.0,2.3,0.00,0.0,1.0,0.0,8.4,6.0,7.0,2.0,0.14,0.05,0.009,4.0,49.0,0.0,0.0,0.0,0.0,33.47,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.001,0.000,0.001,0.000,0.000,0.00